In [ ]:
import pandas as pd
import sys

inputfile_1 = sys.argv[1] # raw virion csv
inputfile_2 = sys.argv[2] # representational sequence data
inputfile_3 = sys.argv[3] # length data

# raw data
data_procession = []
with open(inputfile_1,'r', encoding='ISO-8859-1') as f:
    for line in f.read().splitlines():
        data = line.split('\t')
        if len(data) < 34:
            continue
        else:
            if data[2] == '' or data[3] == '' or data[7] == '' or data[33] == '': # delete non host genus, host id, virus id and NCBIAccession data
                continue
            else:
                data_procession.append(data)

data = pd.DataFrame(data_procession[1:-1],columns=data_procession[0])

coro = data[data['VirusFamily'] == 'coronaviridae']

host_info = coro[['HostTaxID', 'HostGenus', 'HostFamily', 'HostOrder', 'HostClass']]
host_info = host_info.drop_duplicates(['HostTaxID'])

# representational data
seq_data = pd.read_csv(inputfile_2)

# length data
seq_length = pd.read_table(inputfile_3, sep='\t')
seq_length.columns = ['Representation', 'Length']

# merge
df_1 = pd.merge(seq_data, seq_length, how='inner', on='Representation')
df_2 = pd.merge(df_1, host_info, how='inner', on='HostTaxID')

df_2.to_csv('host_taxon_info.csv', index=None)

In [3]:
# raw data
data_procession = []
with open('/home/ouconstand/data/Virus_Host/Virion.csv','r', encoding='ISO-8859-1') as f:
    for line in f.read().splitlines():
        data = line.split('\t')
        if len(data) < 34:
            continue
        else:
            if data[2] == '' or data[3] == '' or data[7] == '' or data[33] == '': # delete non host genus, host id, virus id and NCBIAccession data
                continue
            else:
                data_procession.append(data)

data = pd.DataFrame(data_procession[1:-1],columns=data_procession[0])

In [6]:
data.head()

,Host,Virus,HostTaxID,VirusTaxID,HostNCBIResolved,VirusNCBIResolved,ICTVRatified,HostGenus,HostFamily,HostOrder,...,PMID,ReleaseYear,ReleaseMonth,ReleaseDay,CollectionYear,CollectionMonth,CollectionDay,AssocID,DatabaseDOI,NCBIAccession
0,,adenovirus predict_adv-13,27659,2710946,TRUE,TRUE,FALSE,sturnira,phyllostomidae,chiroptera,...,,2020,3,1,2011,11,20,,,MK325782.1
1,,adenovirus predict_adv-13,27659,2710946,TRUE,TRUE,FALSE,sturnira,phyllostomidae,chiroptera,...,,2021,3,9,2011,11,1,,,MK325782
2,,adenovirus predict_adv-19,10111,2710952,TRUE,TRUE,FALSE,praomys,muridae,rodentia,...,,2020,3,1,2009,4,1,,,MK325792.1
3,,adenovirus predict_adv-19,10111,2710952,TRUE,TRUE,FALSE,praomys,muridae,rodentia,...,,2021,3,9,2009,4,1,,,MK325792
4,,adenovirus predict_adv-21,10111,2710955,TRUE,TRUE,FALSE,praomys,muridae,rodentia,...,,2020,3,1,2010,1,5,,,MK325833.1


In [11]:
seq_data.head()

,VirusTaxID,HostTaxID,Representation
0,694014,8835,MF034372.1
1,694014,8835,MF034361.1
2,694014,8835,MF034370.1
3,694014,8835,JX173489.1
4,694014,8835,DQ401286.1


In [21]:
len(set(seq_data['VirusTaxID']))

942

In [12]:
seq_length.head()

,Representation,Length
0,MW320914.1,29742
1,MW320920.1,29506
2,MW320940.1,29507
3,MW320956.1,29515
4,MW320961.1,29782


In [13]:
final = pd.merge(seq_data, seq_length, how='inner', on='Representation')

In [14]:
final.head()

,VirusTaxID,HostTaxID,Representation,Length
0,694014,8835,MF034372.1,1596
1,694014,110668,MF034372.1,1596
2,694014,48849,MF034372.1,1596
3,694014,9031,MF034372.1,1596
4,694014,8835,MF034361.1,561


In [66]:
train = final[['VirusTaxID', 'Representation', 'Length']]
print(train.shape)
train.drop_duplicates(subset=['VirusTaxID', 'Representation', 'Length'], inplace=True)
print(train.shape)

(59759, 3)
(59205, 3)


<ipython-input-66-57804715d2eb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop_duplicates(subset=['VirusTaxID', 'Representation', 'Length'], inplace=True)


In [73]:
less_20k = train[train['Length'] < 20000]
more_20k = train[train['Length'] > 20000]

more_20k_id = set(more_20k['VirusTaxID'])
less_20k_id = set(less_20k['VirusTaxID'])

info = []

for i in less_20k_id:
    if i not in more_20k_id:
        info.append(i)


In [77]:
with open('/mnt/c/Users/ouyangkang/Desktop/virustaxid.txt', 'w') as f:
    f.writelines(map(lambda x:str(x)+'\n' ,info))

In [29]:
train

,VirusTaxID,Representation,Length
0,694014,MF034372.1,1596
4,694014,MF034361.1,561
5,694014,MF034370.1,561
6,694014,JX173489.1,1595
9,694014,DQ401286.1,137
...,...,...,...
59754,2042560,KX285747.1,387
59755,2042560,KX284997.1,249
59756,2042560,KX286039.1,286
59757,694006,MT064734.1,261


In [43]:
repre = train.groupby('Representation')['VirusTaxID']
repre

In [47]:
repre.groups

{'AB007451.1': [59492], 'AB514164.1': [59472], 'AB514461.1': [4430, 4431], 'AB535528.1': [57583], 'AB543561.1': [57489], 'AB548621.1': [1193], 'AB548624.1': [56263], 'AB551247.1': [59089], 'AB555559.1': [1378], 'AB555560.1': [1379], 'AB644273.1': [59021], 'AB644274.1': [59022], 'AB683970.1': [59263], 'AB683971.1': [58108], 'AB781788.1': [1654], 'AB781790.1': [1056], 'AB781791.1': [1057], 'AB781796.1': [1656], 'AB781798.1': [1060], 'AB781800.1': [470], 'AB781805.1': [1061], 'AB781806.1': [1657], 'AB781808.1': [1063], 'AB781812.1': [1064], 'AB781815.1': [1065], 'AB812842.1': [55645], 'AB812844.1': [55646], 'AB812854.1': [55648], 'AB812859.1': [55649], 'AB812860.1': [55650], 'AB812863.1': [55651], 'AB812864.1': [55652], 'AB812868.1': [55653], 'AB812870.1': [55654], 'AB812871.1': [55655], 'AB812872.1': [55656], 'AB857233.1': [59507], 'AB857234.1': [56358], 'AB857235.1': [56359], 'AB889995.1': [55925], 'AB907624.1': [1672], 'AB907631.1': [1070], 'AB907634.1': [1673], 'AB918717.1': [57501], 

In [58]:
dic_repre = repre.groups
n = 0
for k, v in dic_repre.items():
    if len(v) > 1:
        n += 1

print(n)

254


In [61]:
x = pd.DataFrame(train.groupby('Representation')['VirusTaxID'].count())
x = x.reset_index()
x.columns = ['representation', 'count']

In [64]:
x[x['count'] >= 2]

,representation,count
2,AB514461.1,2
44,AB918719.1,2
68,AF220295.1,2
83,AM261815.1,2
85,AM261821.1,2
...,...,...
12211,MW600658.1,8
12212,MW600666.1,4
24143,MW802581.1,4
27680,MW898904.1,2


In [ ]:
train.groupby('VirusTaxID')

In [6]:
import pandas as pd

repre_data = pd.read_csv('/home/ouconstand/data/Virus_Host/repre_data.csv')
repre_data.columns = ['VirusTaxID', 'HostTaxID', 'Accession']
len_data = pd.read_table('/home/ouconstand/data/Virus_Host/seq_length.tsv', sep='\t')
#len_data = len_data.reset_index()
len_data.columns = ['Accession', 'Length']

In [7]:
len_data.head()

,Accession,Length
0,OE996255.1,29224
1,OE996275.1,29663
2,OE996297.1,29834
3,OE996857.1,29644
4,OE996881.1,29804


In [8]:
repre_data.head()

,VirusTaxID,HostTaxID,Accession
0,694014,8835,MF034372.1
1,694014,8835,AJ311317.1
2,694014,8835,MT984601.1
3,694014,8835,MK581207.1
4,694014,8835,MK581202.1


In [10]:
train = pd.merge(repre_data, len_data, how='inner', on='Accession')

In [11]:
len(set(train['VirusTaxID']))

942

In [12]:
train

,VirusTaxID,HostTaxID,Accession,Length
0,694014,8835,MF034372.1,1596
1,694014,110668,MF034372.1,1596
2,694014,9031,MF034372.1,1596
3,694014,48849,MF034372.1,1596
4,694014,8835,AJ311317.1,27635
...,...,...,...,...
50085,694014,30538,OM912702.1,27694
50086,2042586,302402,KX285253.1,387
50087,2042586,302402,KX285866.1,289
50088,694005,435406,MT064772.1,269


In [19]:
more_25 = train[train['Length'] >= 25000]
len(set(more_25['VirusTaxID']))

531

In [18]:
more_tax = pd.DataFrame(train.groupby('Accession')['VirusTaxID'].count())
more_tax = more_tax.reset_index()
more_tax.columns = ['Accession', 'Count']
more_tax[more_tax['Count'] >= 2]

,Accession,Count
1,AB539081.1,4
15,AB918718.1,3
18,AF220295.1,7
21,AJ311317.1,2
30,AY394984.1,2
...,...,...
40229,OU077605.1,10
40239,OU078223.1,2
48531,OV039277.1,10
48547,OV264044.1,3


In [20]:
more_25.to_csv('/mnt/c/Users/ouyangkang/Desktop/more_than_25k.csv', index=None)